In [1]:
import brightway2 as bw
import bw2data as bd
import lxml.builder
import networkx as nx
from lxml.builder import ElementMaker
from lxml.etree import tostring
#import pygephi


In [2]:
# Load the required project and select the desired database
bw.projects.set_current("staging")
selected_database = bw.Database("materials2")

### Using the inbuilt gexf function to generate the graph of the database
This is not the recommended method because of the limitations with the built-in functions.

In [ ]:
gexfgraph = bw.DatabaseToGEXF("biosphere3")
#gexfgraph.get_data(E=lxml.builder.ElementMaker)
gexfgraph.export()
#gexfgraph.get_data(E=ElementMaker)

### Use this section only for debugging

To check and debug different parts of the activities, exchanges, and their key-value pairs.

In [5]:
for activity in selected_database:

    if 'worksheet name' not in activity:
        print("activity is", activity)
        for exchange in activity.exchanges():
            #print("exchange is", exchange.as_dict(),"\n")
            if exchange.as_dict()['type']=='technosphere':
                print('Technosphere Input \n')
                db_name = bw.Database(exchange.as_dict()['input'][0])
                print(db_name.get(exchange.as_dict()['input'][1])["name"], exchange.as_dict()['amount'], db_name.get(exchange.as_dict()['input'][1])["unit"], db_name.get(exchange.as_dict()['input'][1])["location"] )
            if exchange.as_dict()['type']=='production':
                print('Production output\n')
                db_name_p = bw.Database(exchange.as_dict()['output'][0])
                print(db_name_p.get(exchange.as_dict()['output'][1])["name"],exchange.as_dict()['amount'],db_name_p.get(exchange.as_dict()['output'][1])["unit"],db_name_p.get(exchange.as_dict()['output'][1])["location"])



activity is 'Pyrolytic carbon' (kilogram, GLO, None)
Production output

Pyrolytic carbon 0.65 kilogram GLO
Technosphere Input 

sawing, softwood 1 kilogram Europe without Switzerland
activity is 'Char production' (unit, GLO, None)
Production output

Char production 1 unit GLO
Technosphere Input 

bamboo forestry, sustainable forest management 1 kilogram CN


In [7]:
import exchanges_from_activities as efa
for activity in selected_database:

    if 'worksheet name' not in activity:
        print("activity is", activity)
        for exchange in activity.exchanges():
            #print("exchange is", exchange.as_dict(),"\n")
            if exchange.as_dict()['type']=='technosphere':
                print('Technosphere Input \n')
                db_name = exchange.as_dict()['input'][0]
                exchange_key=exchange.as_dict()['input'][1]
                exchange_name, exchange_unit, exchange_location = efa.invoke_exchanges(db_name,exchange_key)
                print(exchange_name, exchange.as_dict()['amount'], exchange_unit, exchange_location )
            if exchange.as_dict()['type']=='production':
                print('Production output\n')
                db_name = exchange.as_dict()['output'][0]
                exchange_key=exchange.as_dict()['output'][1]
                exchange_name, exchange_unit, exchange_location = efa.invoke_exchanges(db_name,exchange_key)
                print(exchange_name, exchange.as_dict()['amount'], exchange_unit, exchange_location )




activity is 'Pyrolytic carbon' (kilogram, GLO, None)
Production output

Pyrolytic carbon 0.65 kilogram GLO
Technosphere Input 

sawing, softwood 1 kilogram Europe without Switzerland
activity is 'Char production' (unit, GLO, None)
Production output

Char production 1 unit GLO
Technosphere Input 

bamboo forestry, sustainable forest management 1 kilogram CN


In [89]:
for activity in selected_database:

    print(activity)
    for exchange in activity.exchanges():
        if exchange.as_dict()["type"] == "biosphere":
            print ("biosphhere exists \n", exchange.as_dict())
            biodb = bw.Database("biosphere3")
            print(biodb.get(exchange.as_dict()['input'][1]))
            print(biodb.get(exchange.as_dict()['input'][1])["name"])
            print(biodb.get(exchange.as_dict()['input'][1])["unit"])
            print(biodb.get(exchange.as_dict()['input'][1])["type"])


'AFRP production' (kilogram, GLO, None)
biosphhere exists 
 {'output': ('materials2', 'fab05c17548a44a0a7852635c994794b'), 'input': ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'), 'amount': 1, 'type': 'biosphere'}
'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))
Carbon dioxide, fossil
kilogram
emission
'GFRP production' (kilogram, GLO, None)
biosphhere exists 
 {'output': ('materials2', 'cd16b5591d9e48f3a3ad7bed69a7f250'), 'input': ('biosphere3', '50f528ca-45e7-42d0-b399-998ea63ddabf'), 'amount': 1, 'type': 'biosphere'}
'Methane, bromochlorodifluoro-, Halon 1211' (kilogram, None, ('air', 'lower stratosphere + upper troposphere'))
Methane, bromochlorodifluoro-, Halon 1211
kilogram
emission
'Ti64 production' (kilogram, GLO, None)
biosphhere exists 
 {'output': ('materials2', '0508a9434cd24602a9e74cdfd0f70e34'), 'input': ('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'), 'amount': 1, 'type': 'biosphere'}
'Carbon dioxide, fossil' (kilogram, None,

### Convert to graphml file.
Convert the activities, the products, technosphere and biosphere flows into an interconnected network diagram. This resultant file is stored in the graphml file format. The "draw" functins of networkx are used because graphml allows much more flexibility in terms of export and editing using other open-source graphml editing programs such as gephi and cytoscape.

In [35]:
#Declare colors and other parameters required for the various nodes (activities, technosphere flows etc.) and edges/lines here.

    #Colors for nodes and edges
activity_node_color = "orange"
production_node_color = 'black'
production_edge_color = 'black'
technosphere_node_color = 'purple'
technosphere_edge_color = 'purple'
biosphere_node_color ='green'
biosphere_edge_color = 'green'

    #line types for nodes and edges
production_line_style='solid'
technosphere_line_style='solid'
biosphere_line_style='dashed'

In [36]:
graph = nx.DiGraph()

for activity in selected_database:


    #print(activity['name'], activity['unit'], activity['type'], activity['location']) # for debugging only

    # Iterate over each exchange
    for exchange in activity.exchanges():

        #Add the parent activity as a node. Repeated parent activities (as the loop iterates)  will be ignored by networkX
        
        graph.add_node(activity['name'], label="Activity - " + activity['name'], amount=0,unit=activity['unit'], location=activity['location'],type=activity['type'], from_database=activity['database'],  color=activity_node_color)
        
        #Production 'output' are added as outputs from the parent activity

        if exchange.as_dict()['type'] =='production':
            production_display_name = exchange.as_dict()["reference product"] #+ "-" + str(exchange.as_dict()["amount"]) + " " + exchange.as_dict()["unit"]
            
            #add a node corresponding to the exchange
            graph.add_node(production_display_name, label=exchange.as_dict()['reference product'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=production_node_color)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(activity['name'], production_display_name, color=production_edge_color, line_style=production_line_style, arrows=True)

        #Technosphere 'input' exchanges to the parent activity are added

        elif exchange.as_dict()['type'] =='technosphere' :

            #add a node corresponding to the exchange
            graph.add_node(exchange.as_dict()['reference product'], label=exchange.as_dict()['name'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=technosphere_node_color)

            #add the arrow/edge connecting this exchange to the parent activity "node"
            graph.add_edge(exchange.as_dict()['reference product'], activity['name'], color=technosphere_edge_color,  line_style=technosphere_line_style, arrows=True)
            
        #Biosphere 'output' exchanges from the parent activity are added to the graph
        elif exchange.as_dict()['type'] =='biosphere':

            #Only the key, amount and type of the bioshpere exchange are present in the exchange.as_dict(). Therefore to get the, name of exchange, the key must be used to invoke the name from the cocnerned biosphere database. The key is present as a tuple of the form - 'input': ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9').
            biodb=bw.Database("biosphere3")
            name_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['name']
            unit_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['unit']

            #biosphere category is a tuple in the form (air, urban air close to ground). Therefore, we store it as string before referring to it as label and location in the corresponding node.
            biosphere_category = biodb.get(exchange.as_dict()['input'][1])["categories"][0]
            biosphere_category_detail = biodb.get(exchange.as_dict()['input'][1])["categories"][0]

            # the node and edges/arrows are added to the graph
            graph.add_node(name_biosphere_exchange, label=biosphere_category, amount=exchange.as_dict()['amount'], unit=unit_biosphere_exchange, location=biosphere_category_detail, type=exchange.as_dict()['type'],from_database="biosphere",  color=biosphere_node_color)

            graph.add_edge(activity['name'],name_biosphere_exchange, color=biosphere_edge_color, line_style=biosphere_line_style, arrows=True)
        # Add edges to the graph connecting the activities
       # if exchange.as_dict()['type'] =='technosphere':
        #        

        #print('next activity')
# Export the graph to GEXF format, if requird
#nx.write_gexf(graph, "my_graph4.gexf")

#Export to graphml format
nx.write_graphml(graph,'test11.graphml', prettyprint=True)

#just for debugging, we can make a live plot. Note: Do not use this for except for debugging because the figure is immutable.
nx.draw(graph,  with_labels=True, node_shape='s', node_color='black', edge_color='blue')

KeyError: 'reference product'

In [ ]:
# implementation with external function
graph = nx.DiGraph()

for activity in selected_database:


    # Iterate over each exchange
    for exchange in activity.exchanges():

        #Add the parent activity as a node. Repeated parent activities (as the loop iterates)  will be ignored by networkX
        
        graph.add_node(activity['name'], label="Activity - " + activity['name'], amount=0,unit=activity['unit'], location=activity['location'],type=activity['type'], from_database=activity['database'],  color=activity_node_color)
        

    # For the case where the key 'worksheet name' is not in activity, which indicates that the activity is newly created in the database, and not copied from another database.

        if 'worksheet name' not in activity:
            
            # The three types of exchanges - production, technosphere and biosphere - are added as nodes and connected.
            if exchange.as_dict()['type'] =='production':
                db_name = exchange.as_dict()['output'][0]
                exchange_key=exchange.as_dict()['output'][1]
                exchange_name, exchange_unit, exchange_location = efa.invoke_exchanges(db_name,exchange_key)
                exchange_amount=exchange.as_dict()['amount']

                #add a node corresponding to the exchange
                graph.add_node(production_display_name, label=exchange.as_dict()['reference product'], amount=exchange_amount, unit=exchange_unit, location=exchange_location, type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=production_node_color)

                #add the arrow/edge connecting this exchange to the parent activity "node"
                graph.add_edge(activity['name'], production_display_name, color=production_edge_color, line_style=production_line_style, arrows=True)               
            if exchange.as_dict()['type'] =='technosphere':


        else: # The case of activities copied over from other databases or projects.

        #Production 'output' are added as outputs from the parent activity

            if exchange.as_dict()['type'] =='production':
                production_display_name = exchange.as_dict()["reference product"] #+ "-" + str(exchange.as_dict()["amount"]) + " " + exchange.as_dict()["unit"]
                
                #add a node corresponding to the exchange
                graph.add_node(production_display_name, label=exchange.as_dict()['reference product'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=production_node_color)

                #add the arrow/edge connecting this exchange to the parent activity "node"
                graph.add_edge(activity['name'], production_display_name, color=production_edge_color, line_style=production_line_style, arrows=True)

            #Technosphere 'input' exchanges to the parent activity are added

            elif exchange.as_dict()['type'] =='technosphere' :

                #add a node corresponding to the exchange
                graph.add_node(exchange.as_dict()['reference product'], label=exchange.as_dict()['name'], amount=exchange.as_dict()['amount'], unit=exchange.as_dict()['unit'], location=exchange.as_dict()['location'], type=exchange.as_dict()['type'],from_database=exchange.as_dict()['database'],  color=technosphere_node_color)

                #add the arrow/edge connecting this exchange to the parent activity "node"
                graph.add_edge(exchange.as_dict()['reference product'], activity['name'], color=technosphere_edge_color,  line_style=technosphere_line_style, arrows=True)
                
            #Biosphere 'output' exchanges from the parent activity are added to the graph
            elif exchange.as_dict()['type'] =='biosphere':

                #Only the key, amount and type of the bioshpere exchange are present in the exchange.as_dict(). Therefore to get the, name of exchange, the key must be used to invoke the name from the cocnerned biosphere database. The key is present as a tuple of the form - 'input': ('biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9').
                biodb=bw.Database("biosphere3")
                name_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['name']
                unit_biosphere_exchange = biodb.get(exchange.as_dict()['input'][1])['unit']

                #biosphere category is a tuple in the form (air, urban air close to ground). Therefore, we store it as string before referring to it as label and location in the corresponding node.
                biosphere_category = biodb.get(exchange.as_dict()['input'][1])["categories"][0]
                biosphere_category_detail = biodb.get(exchange.as_dict()['input'][1])["categories"][0]

                # the node and edges/arrows are added to the graph
                graph.add_node(name_biosphere_exchange, label=biosphere_category, amount=exchange.as_dict()['amount'], unit=unit_biosphere_exchange, location=biosphere_category_detail, type=exchange.as_dict()['type'],from_database="biosphere",  color=biosphere_node_color)

                graph.add_edge(activity['name'],name_biosphere_exchange, color=biosphere_edge_color, line_style=biosphere_line_style, arrows=True)
        # Add edges to the graph connecting the activities
       # if exchange.as_dict()['type'] =='technosphere':
        #        

        #print('next activity')
# Export the graph to GEXF format, if requird
#nx.write_gexf(graph, "my_graph4.gexf")

#Export to graphml format
nx.write_graphml(graph,'test11.graphml', prettyprint=True)

#just for debugging, we can make a live plot. Note: Do not use this for except for debugging because the figure is immutable.
nx.draw(graph,  with_labels=True, node_shape='s', node_color='black', edge_color='blue')